In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Define functions

def rfeFeature(indep_X,dep_Y,n):
    
        rfelist=[]
        SFSlist=[]
        
        svc_model = SVC(kernel = 'linear', random_state = 0)
        
        rfemodellist=[svc_model]
        
        for i in   rfemodellist:
            log_rfe = RFE(i, n_features_to_select=n)
            log_fit = log_rfe.fit(indep_X, dep_Y)
            log_rfe_feature=log_fit.transform(indep_X)
            rfelist.append(log_rfe_feature)
            op=log_rfe.get_feature_names_out(input_features=None)
            SFSlist.append(op)
        return rfelist, SFSlist

def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test
    
def cm_prediction(classifier,X_test):
        y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report 
        
        Accuracy=accuracy_score(y_test, y_pred )
        
        report=classification_report(y_test, y_pred)
        return  classifier,Accuracy,report,X_test,y_test,cm

def random(X_train,y_train,X_test):
        
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    

def rfe_classification(accrf): 
    
    rfedataframe=pd.DataFrame(index=['SVC'],columns=['Random'])
                                                                                

    for number,idex in enumerate(rfedataframe.index):
        rfedataframe['Random'][idex]=accrf[number]
    return rfedataframe

In [2]:
dataset1=pd.read_csv("prep.csv",index_col=None)

df2=dataset1

In [3]:
df2

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [4]:
df2 = pd.get_dummies(df2, drop_first=True)

In [5]:
indep_X = df2.drop('classification_yes', axis=1)
dep_Y = df2['classification_yes']

In [6]:
rfelist, SFSlist=rfeFeature(indep_X,dep_Y,6)  
accrf=[]

In [7]:
SFSlist

[array(['al', 'sg_c', 'sg_d', 'rbc_normal', 'dm_yes', 'appet_yes'],
       dtype=object)]

In [8]:
rfelist

[array([[3., 1., 0., 1., 0., 1.],
        [2., 1., 0., 1., 0., 1.],
        [1., 0., 0., 1., 0., 1.],
        ...,
        [3., 1., 0., 1., 1., 0.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 0., 1.]])]

In [9]:
for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)   
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
    accrf.append(Accuracy)

In [10]:
result=rfe_classification(accrf)

C:\Users\swath\AppData\Local\Temp\ipykernel_17524\2863763793.py:79: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rfedataframe['Random'][idex]=accrf[number]


In [11]:
result
#k=6

,Random
SVC,0.99


In [12]:
selected_columns = ['al', 'sg_c', 'sg_d', 'rbc_normal', 'dm_yes', 'appet_yes']  # Replace with your actual column names
indep_x = df2[selected_columns]
scx = StandardScaler()
indsc_x = scx.fit_transform(indep_x)
indsc_x

import pickle
scaler_file="scaler_x.pkl"
pickle.dump(scx,open(scaler_file,"wb"))
SCX=pickle.load(open("scaler_x.pkl",'rb'))

In [13]:
import pickle

In [14]:
filename="RFE_alg_final.sav"
pickle.dump(classifier,open(filename,"wb"))

In [15]:
loaded_model=pickle.load(open("RFE_alg_final.sav",'rb'))

In [16]:
Preinput=SCX.transform([[3,1,0,1,0,1]])

C:\Users\swath\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [17]:
Preinput

array([[ 1.59943471,  1.93649167, -0.48112522,  0.36540762, -0.71910379,
         0.51250193]])

In [18]:
result=loaded_model.predict(Preinput)
#HOW VARIABLES USE IT THAT  NO OF OUTPUT 

In [19]:
result

array([ True])

In [20]:
result.astype(int)

array([1])